# Travel Data Analysis: An End-to-End ETL Process

## Introduction

This notebook outlines a comprehensive data processing pipeline, from extraction to loading (ETL), applied to travel-related data stored in an Excel file named 'travel.xlsx'. The primary objective is to clean, transform, analyze, and finally export the data to derive insights into vehicle performance, associated maintenance costs, and overall profitability. The process involves reading data from multiple sheets, applying various data cleaning and transformation techniques, merging the data, performing calculations for key metrics, validating the data, and saving the results in accessible formats (Excel and CSV).

## Setup and Data Loading

This section focuses on setting up the environment by importing necessary libraries and loading the raw data from the specified Excel file into a pandas DataFrame.

In [12]:
import numpy as np
import pandas as pd

# Setting display options for easier readability
pd.set_option("display.max_columns", None)

# Load data from all sheets of the Excel file
excel_file = pd.ExcelFile("/content/TripWise.xlsx")
sheet_names = excel_file.sheet_names

# Read all sheets into a dictionary of DataFrames
df_dict = {sheet_name: excel_file.parse(sheet_name) for sheet_name in sheet_names}

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


## Data Cleaning and Transformation

In this crucial stage, we focus on cleaning and transforming the data within each DataFrame. This involves several steps: removing duplicate entries to ensure data uniqueness, standardizing column names for consistency and ease of access, handling missing values by imputing with the mean, and potentially applying custom functions to data using `.applymap()` if needed for specific transformations.

**Key Operations:**

- **`.drop_duplicates()`**: Removes rows that are exact duplicates across all columns.
- **`.rename()`**: Although not explicitly used for standardization here, it's a key method for renaming columns or index labels. We achieve standardization by directly assigning a new list of column names.
- **`.fillna()`**: Replaces missing (NaN) values with a specified value or method, in this case, the mean of the column.
- **`.applymap()`**: Applies a function element-wise to a DataFrame. While not strictly necessary for this dataset's initial cleaning, it's a powerful tool for custom transformations.

In [13]:
# Clean and transform each DataFrame
cleaned_df_dict = {}
for sheet_name, df in df_dict.items():
    # 1. Remove duplicates
    df_cleaned = df.drop_duplicates()

    # 2. Standardize column names (lowercase and remove leading/trailing spaces)
    df_cleaned.columns = df_cleaned.columns.str.strip().str.lower()
    # Potential use of .rename() for more complex renaming:
    # df_cleaned = df_cleaned.rename(columns={'Old Name': 'new_name'})


    # 3. Fill missing numerical values with the mean
    # Identify numerical columns
    numeric_cols = df_cleaned.select_dtypes(include=np.number).columns
    df_cleaned[numeric_cols] = df_cleaned[numeric_cols].fillna(df_cleaned[numeric_cols].mean())

    # 4. Applymap (Example: convert all string values to lowercase, if any)
    # For demonstration, let's assume we want to ensure all string entries are lowercase
    # This might not be relevant for all columns but demonstrates the usage
    # df_cleaned = df_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)


    cleaned_df_dict[sheet_name] = df_cleaned

# Display the first few rows of each cleaned DataFrame and check for missing values
for sheet_name, df in cleaned_df_dict.items():
    print(f"--- Cleaned DataFrame: {sheet_name} ---")
    display(df.head())
    print(f"Missing values after cleaning:\n{df.isnull().sum()}")
    print("\n")

--- Cleaned DataFrame: Maintenance ---


/tmp/ipython-input-2364043218.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[numeric_cols] = df_cleaned[numeric_cols].fillna(df_cleaned[numeric_cols].mean())


,maintenance_id,vehicle_id,vehicle_type,service_date,cost,maintenance_cost_per_km,type
0,MT400000,VH101717,Diesel,2024-09-13,5531.72,441.126061,General Service
1,MT400001,VH100208,CNG,2024-01-18,4326.84,391.793367,Oil Change
2,MT400002,VH100141,CNG,2024-07-19,680.26,380.939567,Oil Change
3,MT400003,VH101795,CNG,2024-11-20,9143.39,476.082108,Tyre Replacement
4,MT400004,VH101905,CNG,2024-01-11,7310.94,359.613597,AC Repair


Missing values after cleaning:
maintenance_id             0
vehicle_id                 0
vehicle_type               0
service_date               0
cost                       0
maintenance_cost_per_km    0
type                       0
dtype: int64


--- Cleaned DataFrame: Vehicle ---


,vehicle_id,model,type,purchase_date,base_maintenance_cost,status,city
0,VH100000,Tigor EV,CNG,2022-07-12,84790.75,Active,Pune
1,VH100001,Dzire,Electric,2023-07-19,99952.78,Active,Hyderabad
2,VH100002,Tiago,Diesel,2019-01-22,172364.00,Under Maintenance,Chennai
3,VH100003,Dzire,Diesel,2022-02-18,74600.83,Inactive,Bangalore
4,VH100004,Tigor EV,CNG,2021-09-12,175233.44,Active,Hyderabad


...

Missing values after cleaning:
vehicle_id                 0
maintenance_cost_per_km    0
dtype: int64


--- Cleaned DataFrame: Trips_Per_Vehicle ---


,vehicle_id,total_trips
0,VH100000,5832
1,VH100001,4144
2,VH100002,4626
3,VH100003,5271
4,VH100004,5808


Missing values after cleaning:
vehicle_id     0
total_trips    0
dtype: int64


--- Cleaned DataFrame: Utilization ---


,vehicle_id,total_trips
0,VH100000,0.594859
1,VH100001,0.422685
2,VH100002,0.471848
3,VH100003,0.537638
4,VH100004,0.592411


Missing values after cleaning:
vehicle_id     0
total_trips    0
dtype: int64




## Data Integration: Merging DataFrames

This section focuses on combining the cleaned DataFrames from different sheets into a single, comprehensive DataFrame. We will use the `pd.merge()` function, specifying the common column ('vehicle_id' in this case) and the type of merge (outer merge to keep all records from all DataFrames).

In [15]:
# Merge the cleaned DataFrames based on 'vehicle_id'
# Filter out sheets that do not have 'vehicle_id' column for merging
mergeable_sheets = [sheet_name for sheet_name, df in cleaned_df_dict.items() if 'vehicle_id' in df.columns]

if not mergeable_sheets:
    print("No sheets with 'vehicle_id' found to merge.")
else:
    merged_df = cleaned_df_dict[mergeable_sheets[0]]
    for sheet_name in mergeable_sheets[1:]:
        if 'vehicle_id' in cleaned_df_dict[sheet_name].columns:
            merged_df = pd.merge(merged_df, cleaned_df_dict[sheet_name], on='vehicle_id', how='outer', suffixes=('', '_drop'))
            # Drop duplicate columns that resulted from the merge (if any)
            merged_df.drop([col for col in merged_df.columns if '_drop' in col], axis=1, inplace=True)
        else:
            print(f"Skipping sheet '{sheet_name}' as it does not have a 'vehicle_id' column for merging.")


    # Display the first few rows of the merged DataFrame
    print("--- Merged DataFrame ---")
    display(merged_df.head())
    print(f"Shape of merged DataFrame: {merged_df.shape}")
    print(f"Missing values in merged DataFrame:\n{merged_df.isnull().sum()}")

--- Merged DataFrame ---


,maintenance_id,vehicle_id,vehicle_type,service_date,cost,maintenance_cost_per_km,type,model,purchase_date,base_maintenance_cost,status,city,trip_id,driver_id,driver_name,total_trips,date,month,fare,fuel_cost,payment_type,trip_rating
0,MT401640,VH100000,CNG,2024-12-09,861.95,378.432347,AC Repair,Tigor EV,2022-07-12,84790.75,Active,Pune,TP301748,DR203277,Driver_3277,2322,2024-04-15,4,398.01,83.70,Card,4.14
1,MT401640,VH100000,CNG,2024-12-09,861.95,378.432347,AC Repair,Tigor EV,2022-07-12,84790.75,Active,Pune,TP305931,DR202577,Driver_2577,1146,2024-11-24,11,128.03,29.62,Cash,4.66
2,MT401640,VH100000,CNG,2024-12-09,861.95,378.432347,AC Repair,Tigor EV,2022-07-12,84790.75,Active,Pune,TP308800,DR201842,Driver_1842,2527,2024-09-20,9,428.45,95.61,Card,4.44
3,MT401640,VH100000,CNG,2024-12-09,861.95,378.432347,AC Repair,Tigor EV,2022-07-12,84790.75,Active,Pune,TP310093,DR203918,Driver_3918,2376,2025-06-17,6,126.18,44.92,Card,4.97
4,MT401640,VH100000,CNG,2024-12-09,861.95,378.432347,AC Repair,Tigor EV,2022-07-12,84790.75,Active,Pune,TP311252,DR202987,Driver_2987,1629,2024-01-05,1,162.72,49.15,Card,4.68


Shape of merged DataFrame: (10001465, 22)
Missing values in merged DataFrame:
maintenance_id             0
vehicle_id                 0
vehicle_type               0
service_date               0
cost                       0
maintenance_cost_per_km    0
type                       0
model                      0
purchase_date              0
base_maintenance_cost      0
status                     0
city                       0
trip_id                    0
driver_id                  0
driver_name                0
total_trips                0
date                       0
month                      0
fare                       0
fuel_cost                  0
payment_type               0
trip_rating                0
dtype: int64


## Data Analysis: Calculating Key Metrics

Here, we perform calculations to derive key performance indicators related to maintenance, distance, and profitability. We group the merged data by 'vehicle_id' and use the `.agg()` function to compute the sum of 'maintenance_cost' and 'distance'. We then use `np.divide()` to calculate the 'profit' metric (distance divided by maintenance cost).

**Key Operations:**

- **`.groupby()`**: Groups the DataFrame by one or more columns, enabling aggregated calculations.
- **`.transform()`**: While not used in this specific aggregation, `.transform()` is useful for performing group-specific calculations and broadcasting the result back to the original DataFrame's shape.
- **`np.divide()`**: Performs element-wise division, used here to calculate the profit ratio.

In [17]:
# Calculate maintenance and profit metrics
# Assuming 'cost' is the maintenance cost and calculating 'distance'
# using 'cost' and 'maintenance_cost_per_km' columns
maintenance_metrics = merged_df.groupby('vehicle_id').agg(
    total_maintenance_cost=('cost', 'sum'),
    # Calculate distance from cost and maintenance_cost_per_km
    # Handle potential division by zero
    total_distance=('maintenance_cost_per_km', lambda x: np.sum(merged_df.loc[x.index, 'cost'] / x.replace(0, np.nan))),
).reset_index() # Reset index to make 'vehicle_id' a column

# Calculate profit using np.divide()
# Handle potential division by zero
maintenance_metrics['profit_ratio'] = np.divide(
    maintenance_metrics['total_distance'],
    maintenance_metrics['total_maintenance_cost'],
    out=np.zeros_like(maintenance_metrics['total_distance'], dtype=float),
    where=maintenance_metrics['total_maintenance_cost'] != 0
)


# Display the maintenance and profit metrics
print("--- Maintenance and Profit Metrics ---")
display(maintenance_metrics.head())

--- Maintenance and Profit Metrics ---


,vehicle_id,total_maintenance_cost,total_distance,profit_ratio
0,VH100000,17388027.81,45947.52,0.002642
1,VH100001,13148971.57,34740.00,0.002642
2,VH100002,13340176.10,37318.32,0.002797
3,VH100003,17734148.98,41135.22,0.002320
4,VH100004,14435897.52,45140.48,0.003127


## Vehicle Utilization Analysis

This section assesses how effectively vehicles are utilized. We group the data by 'vehicle_id' to count the number of trips per vehicle. We then calculate a 'trip efficiency' metric by dividing the number of trips by the total distance covered by each vehicle.

In [18]:
# Analyze vehicle utilization (trips count and trip efficiency)
vehicle_utilization = merged_df.groupby('vehicle_id').size().reset_index(name='trips_count')

# Merge with total distance to calculate trip efficiency
vehicle_utilization = pd.merge(vehicle_utilization, maintenance_metrics[['vehicle_id', 'total_distance']], on='vehicle_id', how='left')

# Calculate trip efficiency
# Handle potential division by zero if total_distance is 0
vehicle_utilization['trip_efficiency'] = np.divide(vehicle_utilization['trips_count'], vehicle_utilization['total_distance'], out=np.zeros_like(vehicle_utilization['trips_count'], dtype=float), where=vehicle_utilization['total_distance'] != 0)

# Display vehicle utilization metrics
print("--- Vehicle Utilization Metrics ---")
display(vehicle_utilization.head())

--- Vehicle Utilization Metrics ---


,vehicle_id,trips_count,total_distance,trip_efficiency
0,VH100000,5832,45947.52,0.126927
1,VH100001,4144,34740.00,0.119286
2,VH100002,4626,37318.32,0.123961
3,VH100003,5271,41135.22,0.128138
4,VH100004,5808,45140.48,0.128665


## Data Validation: Checking for Invalid Data

As a final validation step before loading the data, we check the merged DataFrame for any negative or potentially invalid numerical entries. This helps ensure data integrity and identify any anomalies that might have been introduced during the data processing steps. We specifically look for numerical columns and check if any value is less than zero.

In [19]:
# Validate for invalid (e.g., negative) numeric data in the merged DataFrame
numeric_cols_merged = merged_df.select_dtypes(include=np.number).columns
invalid_data_found = False

for col in numeric_cols_merged:
    if (merged_df[col] < 0).any():
        print(f"Invalid (negative) data found in column: {col}")
        invalid_rows = merged_df[merged_df[col] < 0]
        print(f"Sample invalid rows in column {col}:")
        display(invalid_rows.head())
        invalid_data_found = True

if not invalid_data_found:
    print("No invalid (negative) numeric data found in the merged DataFrame.")

No invalid (negative) numeric data found in the merged DataFrame.


## Data Loading and Export

The final stage of the ETL process involves loading the processed and analyzed data into output files. We export the merged DataFrame, maintenance metrics, and vehicle utilization metrics to both an Excel file with separate sheets and individual CSV files for ease of sharing and further analysis.

**Key Operations:**

- **`.to_excel()`**: Exports a DataFrame to an Excel file.
- **`.to_csv()`**: Exports a DataFrame to a CSV file.

In [21]:
# Export the processed data to Excel
output_excel_path = 'processed_travel_data.xlsx'
with pd.ExcelWriter(output_excel_path) as writer:
    # Exporting only the smaller analysis results to Excel due to row limits
    maintenance_metrics.to_excel(writer, sheet_name='Maintenance Metrics', index=False)
    vehicle_utilization.to_excel(writer, sheet_name='Vehicle Utilization', index=False)

print(f"Processed data saved to '{output_excel_path}' (Analysis Metrics only)")

# Export the processed data to CSV files
merged_df.to_csv('merged_travel_data.csv', index=False)
maintenance_metrics.to_csv('maintenance_metrics.csv', index=False)
vehicle_utilization.to_csv('vehicle_utilization.csv', index=False)

print("Processed data saved to individual CSV files.")

Processed data saved to 'processed_travel_data.xlsx' (Analysis Metrics only)
Processed data saved to individual CSV files.


## Conclusion

This notebook successfully implemented an ETL pipeline for the travel data. We started by extracting data from multiple Excel sheets, followed by comprehensive cleaning and transformation steps. The data was then merged, and key metrics related to maintenance, profitability, and vehicle utilization were calculated. Finally, the processed data was validated and loaded into output Excel and CSV files. This structured approach ensures data quality and provides valuable insights for decision-making.